# Libraries & Constants

In [1]:
from neo4j import GraphDatabase
import pandas as pd
import json
import copy
from itertools import chain
from collections import Counter

In [3]:
DATA_MINI_PATH   = "../data/processed/dataset3_mini.csv"
MANUAL_CORRECTED_DATA_MINI_PATH = "../data/processed/json_datasets/corrected_train_data_mini.json"

---

## Load the Data

In [4]:
data_mini = pd.read_csv(DATA_MINI_PATH)
data_mini.head(10)

,City,Category,POI,Description,Country
0,Cairo,sleep,Le Passage,was Iberotel,Egypt
1,Cairo,see,Egyptian Museum,Located in the Midan Tahrir area and officiall...,Egypt
2,Cairo,see,Ibn Tulun,"Arguably the oldest mosque in Cairo, built bet...",Egypt
3,Cairo,see,Al-Azhar Park,A recently opened landscaped gardens overlooki...,Egypt
4,Cairo,see,Khan El Khalily,Cairo's souk area where visitors will find man...,Egypt
5,Cairo,see,Abdeen Palace,Located about one Kilometer away from the Mida...,Egypt
6,Cairo,see,Pharaonic Village,It is about twenty minutes driving from Downtown.,Egypt
7,Cairo,do,The Culture Wheel,The largest independent cultural centre in Cai...,Egypt
8,Cairo,do,The Garden Theater,In Al-Azhar Park offers a range of musical per...,Egypt
9,Cairo,do,Cairo Opera House,It hosted the Cairo International Film Festiva...,Egypt


In [98]:
print(f"# of unique POIs  -> {data_mini.POI.nunique()}")
print(f"# of unique cities  -> {data_mini.City.nunique()}")
print(f"# of unique countries  -> {data_mini.Country.nunique()}")

# of unique POIs  -> 2881
# of unique cities  -> 12
# of unique countries  -> 9


In [5]:
corrected_file = open(MANUAL_CORRECTED_DATA_MINI_PATH)
corrected_data = json.load(corrected_file)
corrected_data[5:10]

[{'document': 'Located about one Kilometer away from the Midan El-Tahrir a five minutes walk, home of the last king of Egypt the exiled king Farouk.',
  'annotation': [[18, 27, 'FAC'], [38, 57, 'FAC']]},
 {'document': 'It is about twenty minutes driving from Downtown.',
  'annotation': [[40, 48, 'LOC']]},
 {'document': 'The largest independent cultural centre in Cairo, offers concerts almost every night.',
  'annotation': []},
 {'document': 'In Al-Azhar Park offers a range of musical performances. Al-Azhar Park is also a great place for an evening stroll.',
  'annotation': []},
 {'document': 'It hosted the Cairo International Film Festival in 2012 and screened some international films with very cheap subsidized ticket prices.',
  'annotation': [[10, 47, 'ORG'], [51, 55, 'DATE']]}]

In [6]:
corrected_data[:5]

[{'document': 'was Iberotel', 'annotation': [[4, 12, 'ORG']]},
 {'document': "Located in the Midan Tahrir area and officially named Museum of Egyptian Antiquities but known by all as the Egyptian Museum, it hosts the world's premier collection of ancient Egyptian artifacts.",
  'annotation': [[15, 27, 'FAC'], [54, 84, 'ORG'], [105, 124, 'ORG']]},
 {'document': 'Arguably the oldest mosque in Cairo, built between 868-884.',
  'annotation': []},
 {'document': 'A recently opened landscaped gardens overlooking the Citadel',
  'annotation': [[53, 60, 'ORG']]},
 {'document': "Cairo's souk area where visitors will find many merchants selling perfume, spices, Gold, Egyptian hand craft.",
  'annotation': []}]

In [7]:
corr_d = copy.deepcopy(corrected_data)
corr_df = pd.DataFrame(corr_d)
corr_df[5:10]

,document,annotation
5,Located about one Kilometer away from the Mida...,"[[18, 27, FAC], [38, 57, FAC]]"
6,It is about twenty minutes driving from Downtown.,"[[40, 48, LOC]]"
7,The largest independent cultural centre in Cai...,[]
8,In Al-Azhar Park offers a range of musical per...,[]
9,It hosted the Cairo International Film Festiva...,"[[10, 47, ORG], [51, 55, DATE]]"


## Create separate dataframe for each entity

In [207]:
def get_ent_rows(df, entity):
    ents = []
    tags = []
    ent_list = []
    tag_list = []
    for i, row in enumerate(df["annotation"]):
        for e in row:
            if e[2] == entity:
                start, end = e[0],e[1]
                ent_list.append(df.iloc[i]["document"][start:end])
                tag_list.append(e[2])

        ents.append(ent_list)
        tags.append(tag_list)
        ent_list = []
        tag_list = []
        
    return ents, tags

In [211]:
org = copy.deepcopy(data_mini)
org["ents"], org["tags"] = get_ent_rows(corr_df, "ORG")

In [212]:
org.head()

,City,Category,POI,Description,Country,ents,tags
0,Cairo,sleep,Le Passage,was Iberotel,Egypt,[Iberotel],[ORG]
1,Cairo,see,Egyptian Museum,Located in the Midan Tahrir area and officiall...,Egypt,"[Museum of Egyptian Antiquities, the Egyptian ...","[ORG, ORG]"
2,Cairo,see,Ibn Tulun,"Arguably the oldest mosque in Cairo, built bet...",Egypt,[],[]
3,Cairo,see,Al-Azhar Park,A recently opened landscaped gardens overlooki...,Egypt,[Citadel],[ORG]
4,Cairo,see,Khan El Khalily,Cairo's souk area where visitors will find man...,Egypt,[],[]


In [221]:
loc = copy.deepcopy(data_mini)
loc["ents"], loc["tags"] = get_ent_rows(corr_df, "LOC")

In [222]:
loc[5:10]

,City,Category,POI,Description,Country,ents,tags
5,Cairo,see,Abdeen Palace,Located about one Kilometer away from the Mida...,Egypt,[],[]
6,Cairo,see,Pharaonic Village,It is about twenty minutes driving from Downtown.,Egypt,[Downtown],[LOC]
7,Cairo,do,The Culture Wheel,The largest independent cultural centre in Cai...,Egypt,[],[]
8,Cairo,do,The Garden Theater,In Al-Azhar Park offers a range of musical per...,Egypt,[],[]
9,Cairo,do,Cairo Opera House,It hosted the Cairo International Film Festiva...,Egypt,[],[]


In [223]:
fac = copy.deepcopy(data_mini)
fac["ents"], fac["tags"] = get_ent_rows(corr_df, "FAC")

In [313]:
fac.head(10)

,City,Category,POI,Description,Country,ents,tags
0,Cairo,sleep,Le Passage,was Iberotel,Egypt,[],[]
1,Cairo,see,Egyptian Museum,Located in the Midan Tahrir area and officiall...,Egypt,[Midan Tahrir],[FAC]
2,Cairo,see,Ibn Tulun,"Arguably the oldest mosque in Cairo, built bet...",Egypt,[],[]
3,Cairo,see,Al-Azhar Park,A recently opened landscaped gardens overlooki...,Egypt,[],[]
4,Cairo,see,Khan El Khalily,Cairo's souk area where visitors will find man...,Egypt,[],[]
5,Cairo,see,Abdeen Palace,Located about one Kilometer away from the Mida...,Egypt,"[Kilometer, the Midan El-Tahrir]","[FAC, FAC]"
6,Cairo,see,Pharaonic Village,It is about twenty minutes driving from Downtown.,Egypt,[],[]
7,Cairo,do,The Culture Wheel,The largest independent cultural centre in Cai...,Egypt,[],[]
8,Cairo,do,The Garden Theater,In Al-Azhar Park offers a range of musical per...,Egypt,[],[]
9,Cairo,do,Cairo Opera House,It hosted the Cairo International Film Festiva...,Egypt,[],[]


In [225]:
event = copy.deepcopy(data_mini)
event["ents"], event["tags"] = get_ent_rows(corr_df, "EVENT")

In [230]:
event[606:612]

,City,Category,POI,Description,Country,ents,tags
606,Dublin,do,National Concert Hall,Offers classical concerts. Frequent performanc...,Ireland,[],[]
607,Dublin,do,Bord Gáis Energy Theatre,The theatre offers a wide range of shows featu...,Ireland,[],[]
608,Dublin,do,International Dublin Gay Theatre Festival,The International Dublin Gay Theatre Festival ...,Ireland,[],[]
609,Dublin,do,Traditional games at Croke Park Stadium,Catch a '''hurling''' or [http://www.gaa.ie/ '...,Ireland,[the Guinness Book of World Records],[EVENT]
610,Dublin,do,Tallaght Stadium,Watch a Shamrock Rovers F.C. soccer match duri...,Ireland,[],[]
611,Dublin,do,RDS Arena,Occasional home games are played at '''Aviva S...,Ireland,"[Heineken Cup, the Guinness Pro12]","[EVENT, EVENT]"


In [231]:
date = copy.deepcopy(data_mini)
date["ents"], date["tags"] = get_ent_rows(corr_df, "DATE")

In [233]:
date[5:10]

,City,Category,POI,Description,Country,ents,tags
5,Cairo,see,Abdeen Palace,Located about one Kilometer away from the Mida...,Egypt,[],[]
6,Cairo,see,Pharaonic Village,It is about twenty minutes driving from Downtown.,Egypt,[],[]
7,Cairo,do,The Culture Wheel,The largest independent cultural centre in Cai...,Egypt,[],[]
8,Cairo,do,The Garden Theater,In Al-Azhar Park offers a range of musical per...,Egypt,[],[]
9,Cairo,do,Cairo Opera House,It hosted the Cairo International Film Festiva...,Egypt,[2012],[DATE]


In [234]:
money = copy.deepcopy(data_mini)
money["ents"], money["tags"] = get_ent_rows(corr_df, "MONEY")

In [237]:
money[1000:1005]

,City,Category,POI,Description,Country,ents,tags
1000,Las Vegas,sleep,Sam's Town,Large motel property with considerable parking...,USA,[],[]
1001,Las Vegas,other,Elysium Internet Cafe,"$3/hr, or connect to it for free!",USA,[3],[MONEY]
1002,Las Vegas,other,Spring Mountains,"Five peaks over 11,000' which is Bristlecone P...",USA,[],[]
1003,Las Vegas,other,Las Vegas Ski and Snowboard Resort,45 minutes from Las Vegas.,USA,[],[]
1004,Las Vegas,other,Brian Head Resort,Offers more vertical feet but is about a three...,USA,[],[]


In [238]:
# ents = []
# tags = []
# ent_list = []
# tag_list = []
# for i, row in enumerate(corr_df["annotation"]):
#     for e in row:
#         start, end = e[0],e[1]
#         ent_list.append(corr_df.iloc[i]["document"][start:end])
#         tag_list.append(e[2])
# #     print(ent_list)
# #     print(tag_list)
#     ents.append(ent_list)
#     tags.append(tag_list)
#     ent_list = []
#     tag_list = []

In [239]:
# data_mini_modified = copy.deepcopy(data_mini)

In [240]:
# data_mini_modified["ents"] = ents
# data_mini_modified["tags"] = tags

In [241]:
# data_mini_modified

In [306]:
pd.Series(Counter(chain.from_iterable(data_mini_modified["tags"])))

ORG      3765
FAC      1142
LOC       896
DATE     2350
MONEY     280
EVENT     126
dtype: int64

In [244]:
pd.Series(Counter(chain.from_iterable(fac["ents"])))

Midan Tahrir              1
Kilometer                 1
the Midan El-Tahrir       1
Dokki Metro               1
mosques                   5
                         ..
Ramleh Station            3
Cecil Hotel               1
the Alexandria Library    1
the Montazah Palace       1
Grand Plaza               1
Length: 594, dtype: int64

---

## Connection to Neo4j

In [8]:
host = 'bolt://localhost:7687'
user = 'mr'
password = 'mrneo4j'
driver = GraphDatabase.driver(host,auth=(user, password))

In [70]:
# def insert_data(query, rows, batch_size = 500):
#     # Function to handle the updating the Neo4j database in batch mode.
    
#     batch = 0
    
#     while batch * batch_size < len(rows):

#         result = conn.query(query, 
#                          parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
#         batch += 1
    
# #     print(pd.DataFrame([dict(_) for _ in result]))
        
#     return pd.DataFrame([dict(_) for _ in result])

In [9]:
def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

---

## Build Knowledge Graph

In [280]:
#Remove any existing nodes and relations
neo4j_query("""
MATCH (n) DETACH DELETE n;
""")

""


In [281]:
#Create Constraints
neo4j_query("""
CREATE CONSTRAINT poi IF NOT EXISTS ON (poi:POI) ASSERT poi.name IS UNIQUE
""")

neo4j_query("""
CREATE CONSTRAINT city IF NOT EXISTS ON (ci:City) ASSERT ci.name IS UNIQUE
""")

neo4j_query("""
CREATE CONSTRAINT country IF NOT EXISTS ON (co:Country) ASSERT co.name IS UNIQUE
""")

""


In [282]:
#Create POI nodes
query1 = ("""
UNWIND $rows as row
MERGE (poi:POI {name:row.POI}) 
ON CREATE SET poi.category=row.Category

RETURN count(distinct poi) as POIcount
""")

In [283]:
neo4j_query(query1, params = {"rows":data_mini.to_dict('records')})

,POIcount
0,2881


In [284]:
#Create City nodes
query2 = ("""
UNWIND $rows as row
MERGE (ci:City {name:row.City})
RETURN count(distinct ci) as CityCount
""")

In [285]:
neo4j_query(query2, params = {"rows":data_mini.to_dict('records')})

,CityCount
0,12


In [286]:
#Create Country nodes
query3 = ("""
UNWIND $rows as row
MERGE (co:Country {name:row.Country})
RETURN count(distinct co) as CountryCount
""")

In [287]:
neo4j_query(query3, params = {"rows":data_mini.to_dict('records')})

,CountryCount
0,9


In [288]:
#Create in_City relation
query4 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
MATCH (ci:City {name:row.City})
MERGE (poi)-[r1:in_City]->(ci)
RETURN count(distinct r1) as R1Count
""")

In [289]:
neo4j_query(query4, params = {"rows":data_mini.to_dict('records')})

,R1Count
0,2891


In [290]:
#Create in_Country relation
query5 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
MATCH (co:Country {name:row.Country})
MERGE (poi)-[r2:in_Country]->(co)
RETURN count(distinct r2) as R2Count
""")

In [291]:
neo4j_query(query5, params = {"rows":data_mini.to_dict('records')})

,R2Count
0,2887


In [292]:
#Add Description of each POI
query6 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
SET poi.description=row.Description
RETURN poi.name, poi.category, poi.description
""")

In [293]:
neo4j_query(query6, params = {"rows":data_mini.to_dict('records')})

,poi.name,poi.category,poi.description
0,Le Passage,sleep,was Iberotel
1,Egyptian Museum,see,A must for any visitor to Cairo. The Gold Room...
2,Ibn Tulun,see,"Arguably the oldest mosque in Cairo, built bet..."
3,Al-Azhar Park,see,The Al-Azhar Park is overlooking Darb al-Ahmar...
4,Khan El Khalily,see,Cairo's souk area where visitors will find man...
...,...,...,...
2930,Hyatt Regency Sharm El Sheikh,sleep,5 star resort style hotel with 439 rooms and s...
2931,Noria Resort,sleep,It has been built in traditional Roman Style. ...
2932,Ritz Carlton,sleep,"Spa, diving and snorkelling from the hotel."
2933,The Cleopatra Luxury Resort Collection,sleep,"Spa, diving and snorkelling from the hotel."


---

## Add more Properties, Nodes & Relations to the Graph

**Properties of POIs from Entities:**
* DATE: relatedDates
* MONEY: hasCost

**Relations of POIs from Entities:**
* FAC: nearby_Facility
* LOC: nearby_Touristic_Location
* EVENT: related_Event
* ORG: nearby_Organization


### POIs additional properties

In [294]:
#Add relatedDates property of each POI
query7 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
SET poi.relatedDates=row.ents
RETURN poi.name, poi.category, poi.relatedDates
""")

In [295]:
neo4j_query(query7, params = {"rows":date.to_dict('records')})

,poi.name,poi.category,poi.relatedDates
0,Le Passage,sleep,[]
1,Egyptian Museum,see,[]
2,Ibn Tulun,see,[]
3,Al-Azhar Park,see,[today]
4,Khan El Khalily,see,[]
...,...,...,...
2930,Hyatt Regency Sharm El Sheikh,sleep,[]
2931,Noria Resort,sleep,[]
2932,Ritz Carlton,sleep,[]
2933,The Cleopatra Luxury Resort Collection,sleep,[]


In [299]:
#Add hasCost property of each POI
query8 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
SET poi.hasCosts=row.ents
RETURN poi.name, poi.category, poi.hasCosts
""")

In [300]:
neo4j_query(query8, params = {"rows":money.to_dict('records')})

,poi.name,poi.category,poi.hasCosts
0,Le Passage,sleep,[]
1,Egyptian Museum,see,[]
2,Ibn Tulun,see,[]
3,Al-Azhar Park,see,[]
4,Khan El Khalily,see,[]
...,...,...,...
2930,Hyatt Regency Sharm El Sheikh,sleep,[]
2931,Noria Resort,sleep,[]
2932,Ritz Carlton,sleep,[]
2933,The Cleopatra Luxury Resort Collection,sleep,[]


### POIs additional relations

### nearby_Facility Relation

In [303]:
#Create Facility nodes
query9 = ("""
UNWIND $rows as row
UNWIND row.ents as facility
MERGE (fac:Facility {name:facility})
RETURN count(distinct fac) as FacCount
""")

In [304]:
neo4j_query(query9, params = {"rows":fac.to_dict('records')})

,FacCount
0,594


In [311]:
#Create nearby_Facility relation
query10 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
WITH poi, row
UNWIND row.ents as facility
MATCH (fac:Facility {name:facility})
MERGE (poi)-[r3:nearby_Facility]->(fac)
RETURN count(distinct r3) as R3Count
""")

In [312]:
neo4j_query(query10, params = {"rows":fac.to_dict('records')})

,R3Count
0,944


### nearby_Touristic_Location Relation

In [314]:
#Create Location nodes
query11 = ("""
UNWIND $rows as row
UNWIND row.ents as location
MERGE (loc:Location {name:location})
RETURN count(distinct loc) as LocCount
""")

In [315]:
neo4j_query(query11, params = {"rows":loc.to_dict('records')})

,LocCount
0,311


In [318]:
#Create nearby_Touristic_Location relation
query12 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
WITH poi, row
UNWIND row.ents as location
MATCH (loc:Location {name:location})
MERGE (poi)-[r4:nearby_Touristic_Location]->(loc)
RETURN count(distinct r4) as R4Count
""")

In [319]:
neo4j_query(query12, params = {"rows":loc.to_dict('records')})

,R4Count
0,718


### related_Event Relation

In [320]:
#Create Event nodes
query13 = ("""
UNWIND $rows as row
UNWIND row.ents as event
MERGE (eve:Event {name:event})
RETURN count(distinct eve) as EventCount
""")

In [321]:
neo4j_query(query13, params = {"rows":event.to_dict('records')})

,EventCount
0,91


In [322]:
#Create related_Event relation
query14 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
WITH poi, row
UNWIND row.ents as event
MATCH (eve:Event {name:event})
MERGE (poi)-[r5:related_Event]->(eve)
RETURN count(distinct r5) as R5Count
""")

In [323]:
neo4j_query(query14, params = {"rows":event.to_dict('records')})

,R5Count
0,112


### nearby_Organization Relation

In [324]:
#Create Organization nodes
query15 = ("""
UNWIND $rows as row
UNWIND row.ents as organization
MERGE (org:Organization {name:organization})
RETURN count(distinct org) as OrgCount
""")

In [325]:
neo4j_query(query15, params = {"rows":org.to_dict('records')})

,OrgCount
0,2229


In [326]:
#Create nearby_Organization relation
query16 = ("""
UNWIND $rows as row
MATCH (poi:POI {name:row.POI}) 
WITH poi, row
UNWIND row.ents as organization
MATCH (org:Organization {name:organization})
MERGE (poi)-[r6:nearby_Organization]->(org)
RETURN count(distinct r6) as R6Count
""")

In [327]:
neo4j_query(query16, params = {"rows":org.to_dict('records')})

,R6Count
0,2601


---

# Get Package Recommendation

In [132]:
#Query Knowledge Graph
res = neo4j_query("""
MATCH (poi:POI) 
MATCH (ci:City)
MATCH (co:Country)
RETURN poi.name, poi.category, ci.name, co.name
""")
res.head()

,poi.name,poi.category,ci.name,co.name
0,Cairo Opera House,do,Cairo,Egypt
1,Cairo Jazz Club,do,Cairo,Egypt
2,Mohammed Mahmoud Khalil Museum,see,Cairo,Egypt
3,Tahrir Cinema,do,Cairo,Egypt
4,Best Buy - Apple Center,buy,Cairo,Egypt


In [ ]:
# MATCH (a:User {username: 'user6'})-[r]-(b)
# RETURN r, a, b

In [331]:
#Query Knowledge Graph
res1 = neo4j_query("""
MATCH (poi:POI)-[r]-(b) 
MATCH (ci:City)
WHERE poi.category = $Category AND ci.name = $City
RETURN r, poi, b 
""", params = {"City":"Cairo", "Category":"eat"})
res1.head()

,r,poi,b
0,(),"(name, description, relatedDates, hasCosts, ca...",(name)
1,(),"(name, description, relatedDates, hasCosts, ca...",(name)
2,(),"(name, description, relatedDates, hasCosts, ca...",(name)
3,(),"(name, description, hasCosts, relatedDates, ca...",(name)
4,(),"(name, description, hasCosts, relatedDates, ca...",(name)


In [332]:
data_mini["Category"].value_counts()

eat      618
see      540
sleep    508
buy      494
do       325
drink    254
other    189
go         7
Name: Category, dtype: int64

In [343]:
#Query Knowledge Graph
neo4j_query("""
MATCH (ci:City)
MATCH (poi:POI) 
WHERE ci.name = $City AND poi.category = $Category1
WITH ci, collect(poi.name)[0..3] as Food_Locations
MATCH (poi:POI)
WHERE ci.name = $City AND poi.category = $Category2 
RETURN Food_Locations, collect(poi.name)[0..3] as Drink_Locations
""", params = {"City":"Cairo", "Category1":"eat", "Category2":"drink", "Category3":"see", "Category4":"sleep"})

,Food_Locations,Drink_Locations
0,"[Al Omda, Gad, Chocofolie]","[Stiletto Lounge, Bursa, Tawfiqiyya Souq]"


In [348]:
#Get a single day trip plan
neo4j_query("""
MATCH (ci:City)
MATCH (poi:POI) 
WHERE ci.name = $City AND poi.category = $Category1
WITH ci, collect(poi.name)[0..3] as Food_Locations

MATCH (poi:POI)
WHERE ci.name = $City AND poi.category = $Category2 
WITH ci, Food_Locations, collect(poi.name)[0..3] as Drinks_Locations

MATCH (poi:POI)
WHERE ci.name = $City AND poi.category = $Category3 
WITH ci, Food_Locations, Drinks_Locations, collect(poi.name)[0..3] as Sight_Seeing_Locations

MATCH (poi:POI)
WHERE ci.name = $City AND poi.category = $Category4 

RETURN Food_Locations, Drinks_Locations, Sight_Seeing_Locations, collect(poi.name)[0..3] as Accomodations
""", params = {"City":"Cairo", "Category1":"eat", "Category2":"drink", "Category3":"see", "Category4":"sleep"})

,Food_Locations,Drinks_Locations,Sight_Seeing_Locations,Accomodations
0,"[Al Omda, Gad, Chocofolie]","[Stiletto Lounge, Bursa, Tawfiqiyya Souq]","[Egyptian Museum, Ibn Tulun, Al-Azhar Park]","[Le Passage, Concorde Hotel, Havana Hotel]"


In [19]:
#Get a multiday trip plan
neo4j_query("""
MATCH (poi:POI)-[]->(ci:City) 
WHERE ci.name = $City1 AND poi.category = $Category1
WITH ci, collect(poi.name)[0..4] as Alex_Touristic_Locations

MATCH (poi:POI)-[]->(ci:City)
WHERE ci.name = $City1 AND poi.category = $Category11 
WITH Alex_Touristic_Locations, collect(poi.name)[0..1] as Alex_Accommodations

MATCH (poi:POI)-[]->(ci:City) 
WHERE ci.name = $City2 AND poi.category = $Category2 
WITH ci, Alex_Touristic_Locations, Alex_Accommodations, collect(poi.name)[0..4] as Dubai_Touristic_Locations

MATCH (poi:POI)-[]->(ci:City)
WHERE ci.name = $City2 AND poi.category = $Category21 

RETURN Alex_Touristic_Locations, Alex_Accommodations, Dubai_Touristic_Locations, collect(poi.name)[0..1] as Dubai_Accomodations
""", params = {"City1":"Alexandria", "Category1":"see", "Category11":"sleep","City2":"Dubai", "Category2":"see", "Category21":"sleep"})

,Alex_Touristic_Locations,Alex_Accommodations,Dubai_Touristic_Locations,Dubai_Accomodations
0,"[Cemetery of Mostafa Kamel, Kom el-Shouqafa, B...",[Helnan Palestine],"[Jumeirah Beach, Bastakiya District, Dubai Int...",[Raffles Dubai]
